Importing libraries

In [29]:
import numpy as np
import tensorflow as  tf
import random

 BC   u(t,1) = u(t,-1) = 0

Collocation point creation

In [30]:
n_c = 5000  #collocation points

def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res


x_c = Rand(-1, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = np.array(x_c ,dtype='float64')
t_c = np.array(t_c ,dtype='float64')
input_c = np.vstack((x_c, t_c))
input_c = input_c.T
x_bc_u = np.ones((n_c,), dtype='float64')
x_bc_b = - x_bc_u
input_bcu = tf.stack((x_bc_u, t_c), axis = 1)
input_bcb = tf.stack((x_bc_b, t_c), axis = 1)
input_bc = np.vstack((input_bcu, input_bcb))
xt_in = np.vstack((input_c, input_bc))
xt_in = tf.cast(xt_in, dtype='float64')
y_bc = 0.0
global u_bc
u_bc = (y_bc* np.ones((2*n_c,), dtype='float64')).T
u_bc = tf.cast(u_bc, dtype='float64')

In [31]:
print(xt_in)
print(xt_in[:, 0:1])
print(tf.cast(input_bc, dtype='float64'))
print(u_bc)

tf.Tensor(
[[ 0.98451577  0.46200417]
 [ 0.63523842  0.15602168]
 [ 0.14375039  0.54760817]
 ...
 [-1.          0.79680207]
 [-1.          0.51822456]
 [-1.          0.18691565]], shape=(15000, 2), dtype=float64)
tf.Tensor(
[[ 0.98451577]
 [ 0.63523842]
 [ 0.14375039]
 ...
 [-1.        ]
 [-1.        ]
 [-1.        ]], shape=(15000, 1), dtype=float64)
tf.Tensor(
[[ 1.          0.46200417]
 [ 1.          0.15602168]
 [ 1.          0.54760817]
 ...
 [-1.          0.79680207]
 [-1.          0.51822456]
 [-1.          0.18691565]], shape=(10000, 2), dtype=float64)
tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(10000,), dtype=float64)


Architecture ANN

In [32]:
pinn1 = tf.keras.models.Sequential()
pinn1.add(tf.keras.layers.Dense(2,activation='tanh', kernel_initializer='glorot_normal'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh', kernel_initializer='glorot_normal'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh', kernel_initializer='glorot_normal'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh', kernel_initializer='glorot_normal'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh', kernel_initializer='glorot_normal'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh', kernel_initializer='glorot_normal'))

pinn1.add(tf.keras.layers.Dense(1,activation='tanh', kernel_initializer='glorot_normal'))         #output layer

Custom Loss function

In [34]:
def custom_loss_1(arch, xt, xt_bc):
  x= xt[:, 0:1]
  t= xt[:, 1:2]
  def loss(u_pred, u_bc):
    with tf.GradientTape() as t1:
      t1.watch(x)
      t1.watch(t)
      with tf.GradientTape() as t2:
        t2.watch(x)
        xt = tf.stack((x, t), axis=1)
        u = arch(xt)
        u_x = t2.gradient(u, x)
      u_t = t1.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
    f = u_t + u*u_x - (0.01/tf.pi)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    u_pred = tf.cast(u_pred, dtype='float64')
    u_pred = arch(xt_bc)
    loss_2 = tf.reduce_mean(tf.square(u_pred-u_bc))
    return loss_res + loss_2
  return loss


loss_fn = custom_loss_1(pinn1, xt_in, input_bc)
pinn1.compile(optimizer='adam', loss=loss_fn, metrics=['mse'])
pinn1.fit(tf.cast(input_bc, dtype='float64'), u_bc, batch_size=20 ,epochs=1000)



Epoch 1/1000


ValueError: ignored

Test case

In [ ]:
x_test = np.array(random.uniform(-1, 1), dtype='float64')
t_test = np.array(random.uniform(0, 1), dtype='float64')
xt_test = tf.stack((x_test, t_test), axis=1)
u = pinn1(xt_test)